<a href="https://colab.research.google.com/github/chiamaka249/IgboNER/blob/main/igbobert3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

KeyboardInterrupt: ignored

In [ ]:
with open("/content/Iglacuna.txt") as f:
  lacuna_text = f.read()

In [ ]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Successfully uninstalled tensorflow-2.7.0
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-4168yyuj
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-4168yyuj
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 596 kB 7.0 MB/s 
     |████████████████████████████████| 3.3 MB 49.2 MB/s 
     |████████████████████████████████| 61 kB 665 kB/s 
     |████████████████████████████████| 895 kB 71.0 MB/s 
  Created wheel for transformers: filename=transformers-4.13.0.dev0-py3-none-any.whl size=3254235 sha256=2981594d925fc7b680d76029ebe8e80e172b4dc4cc0912487ea480ea42c409b9
  Stored in directory: /tmp/pip-ephem-wheel-cache-_03nrq68/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attemp

In [ ]:
%%time 
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path(".").glob("**/*.txt")]

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

CPU times: user 17.1 s, sys: 1.24 s, total: 18.3 s
Wall time: 5.13 s


In [ ]:
!mkdir igbo_bert3
tokenizer.save_model("igbo_bert3")

['igbo_bert3/vocab.json', 'igbo_bert3/merges.txt']

In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer(
    "./igbo_bert3/vocab.json",
    "./igbo_bert3/merges.txt",
)

In [ ]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [ ]:
tokenizer.encode("Simone gara akwụkwọ taa.").tokens

['<s>', 'Simone', 'Ġgara', 'Ġakwá»¥kwá»į', 'Ġtaa', '.', '</s>']

In [ ]:
# Check that we have a GPU
!nvidia-smi

Thu Dec  2 14:51:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [ ]:
from google.colab import files
files.upload()

KeyboardInterrupt: ignored

In [ ]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./igbo_bert3", max_len=512)

In [ ]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [ ]:
model.num_parameters()
# => 83 million parameters

83504416

In [ ]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer = tokenizer,
    file_path = "/content/Iglacuna.txt",
    block_size = 128
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:125: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,


CPU times: user 32.9 s, sys: 1.18 s, total: 34.1 s
Wall time: 15.5 s


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./igbo_bert3",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 391448
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 18351
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,6.391700
1000,5.337600
1500,4.856400
2000,4.520100
2500,4.309400
3000,4.130600
3500,4.002500
4000,3.885200
4500,3.793900


Step,Training Loss
500,6.391700
1000,5.337600
1500,4.856400
2000,4.520100
2500,4.309400
3000,4.130600
3500,4.002500
4000,3.885200
4500,3.793900
5000,3.719800


Saving model checkpoint to ./igbo_bert3/checkpoint-10000
Configuration saved in ./igbo_bert3/checkpoint-10000/config.json
Model weights saved in ./igbo_bert3/checkpoint-10000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 1h 50min 26s, sys: 42min 54s, total: 2h 33min 21s
Wall time: 2h 32min 54s


TrainOutput(global_step=18351, training_loss=3.5481224906106106, metrics={'train_runtime': 9174.2898, 'train_samples_per_second': 128.004, 'train_steps_per_second': 2.0, 'total_flos': 2.0182020568559616e+16, 'train_loss': 3.5481224906106106, 'epoch': 3.0})

In [ ]:
trainer.save_model("./igbo_bert3")

Saving model checkpoint to ./igbo_bert3
Configuration saved in ./igbo_bert3/config.json
Model weights saved in ./igbo_bert3/pytorch_model.bin


In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./igbo_bert3",
    tokenizer="./igbo_bert3"
)

loading configuration file ./igbo_bert3/config.json
Model config RobertaConfig {
  "_name_or_path": "./igbo_bert3",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.13.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}

loading configuration file ./igbo_bert3/config.json
Model config RobertaConfig {
  "_name_or_path": "./igbo_bert3",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dro

In [ ]:
# The sun <mask>.
# =>

fill_mask("Abụ m Maazị <mask>.") #= okafor/Ọkafọ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.021709423512220383,
  'sequence': 'Abụ m Maazị A.',
  'token': 348,
  'token_str': ' A'},
 {'score': 0.01375269703567028,
  'sequence': 'Abụ m Maazị O.',
  'token': 382,
  'token_str': ' O'},
 {'score': 0.009152992628514767,
  'sequence': 'Abụ m Maazị Ha.',
  'token': 968,
  'token_str': ' Ha'},
 {'score': 0.008712009526789188,
  'sequence': 'Abụ m Maazị M.',
  'token': 398,
  'token_str': ' M'},
 {'score': 0.007407011929899454,
  'sequence': 'Abụ m Maazị Ya.',
  'token': 1240,
  'token_str': ' Ya'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Nwaanyị na <mask> ji na akara.") #= eri
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.11575991660356522,
  'sequence': 'Nwaanyị na ya ji na akara.',
  'token': 289,
  'token_str': ' ya'},
 {'score': 0.07262290269136429,
  'sequence': 'Nwaanyị na e ji na akara.',
  'token': 271,
  'token_str': ' e'},
 {'score': 0.058269668370485306,
  'sequence': 'Nwaanyị na ha ji na akara.',
  'token': 298,
  'token_str': ' ha'},
 {'score': 0.0558326356112957,
  'sequence': 'Nwaanyị na o ji na akara.',
  'token': 273,
  'token_str': ' o'},
 {'score': 0.02874232828617096,
  'sequence': 'Nwaanyị na- ji na akara.',
  'token': 17,
  'token_str': '-'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Chineke ga- ebibikwa ndị niile na- eme ihe <mask>.") #=ọjọọ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.22356781363487244,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe ọma.',
  'token': 508,
  'token_str': ' ọma'},
 {'score': 0.14048273861408234,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe ọjọọ.',
  'token': 717,
  'token_str': ' ọjọọ'},
 {'score': 0.1313692182302475,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe niile.',
  'token': 431,
  'token_str': ' niile'},
 {'score': 0.12606249749660492,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe a.',
  'token': 266,
  'token_str': ' a'},
 {'score': 0.031202223151922226,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe ahụ.',
  'token': 309,
  'token_str': ' ahụ'}]

In [ ]:
fill_mask("O riri <mask>.")

# This is the beginning of a beautiful <mask>.
# =>

[{'score': 0.4518226385116577,
  'sequence': 'O riri ya.',
  'token': 289,
  'token_str': ' ya'},
 {'score': 0.18243154883384705,
  'sequence': 'O riri ha.',
  'token': 298,
  'token_str': ' ha'},
 {'score': 0.031142286956310272,
  'sequence': 'O riri anya.',
  'token': 405,
  'token_str': ' anya'},
 {'score': 0.022876620292663574,
  'sequence': 'O riri aka.',
  'token': 387,
  'token_str': ' aka'},
 {'score': 0.01775292120873928,
  'sequence': 'O riri ihe.',
  'token': 302,
  'token_str': ' ihe'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Nwaanyị na eri <mask> na akara.") #= ji
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.1460145264863968,
  'sequence': 'Nwaanyị na eri ya na akara.',
  'token': 289,
  'token_str': ' ya'},
 {'score': 0.031209174543619156,
  'sequence': 'Nwaanyị na eri aka na akara.',
  'token': 387,
  'token_str': ' aka'},
 {'score': 0.02808999828994274,
  'sequence': 'Nwaanyị na eri ahụ na akara.',
  'token': 309,
  'token_str': ' ahụ'},
 {'score': 0.016127975657582283,
  'sequence': 'Nwaanyị na eri ihe na akara.',
  'token': 302,
  'token_str': ' ihe'},
 {'score': 0.015274378471076488,
  'sequence': 'Nwaanyị na eri ha na akara.',
  'token': 298,
  'token_str': ' ha'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Nwaanyị na eri ji na <mask>.") #= akara
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.10732906311750412,
  'sequence': 'Nwaanyị na eri ji na ya.',
  'token': 289,
  'token_str': ' ya'},
 {'score': 0.02028745226562023,
  'sequence': 'Nwaanyị na eri ji na-.',
  'token': 17,
  'token_str': '-'},
 {'score': 0.016717415302991867,
  'sequence': 'Nwaanyị na eri ji na jw.',
  'token': 4467,
  'token_str': ' jw'},
 {'score': 0.014438007026910782,
  'sequence': 'Nwaanyị na eri ji na Betel.',
  'token': 2335,
  'token_str': ' Betel'},
 {'score': 0.013661667704582214,
  'sequence': 'Nwaanyị na eri ji na Jeruselem.',
  'token': 1083,
  'token_str': ' Jeruselem'}]

In [ ]:
import shutil

In [ ]:
shutil.make_archive("/content/igbo_bert3", 'zip', "igbo_bert3")

'/content/igbo_bert3.zip'

In [ ]:
from google.colab import files
files.download("/content/igbo_bert3.zip")

FileNotFoundError: ignored